In [4]:
#Import libraries
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn import metrics, svm, naive_bayes
from PIL import Image

In [2]:
#Function to extract labels from files
def extract_labels(file_path):
    with open(file_path) as file_labels:

        file_lines = file_labels.readlines()

        file_shape_labels= [None] * len(file_lines)
        file_position_labels = [None] * len(file_lines)
        file_phoneme_labels = [None] * len(file_lines)
        file_second_phoneme_label = [None] * len(file_lines)
        
        for i, line in enumerate(file_lines):
            file_line = line.split()

            file_shape_labels[i]  = int(file_line[1])
            file_position_labels[i] = int(file_line[2])
            file_phoneme_labels[i] = int(file_line[3])
            file_second_phoneme_label[i] = int(file_line[4])
            
    return np.array(file_shape_labels), np.array(file_position_labels), np.array(file_phoneme_labels), np.array(file_second_phoneme_label)

In [3]:
#Funtion to randomly shuffle data
def shuffle_data(images, shape_labels, position_labels, phoneme_labels, second_phoneme_labels):
    #index = np.arange(len(labels))
    #np.shuffle(index)
    index = np.random.choice(np.arange(len(phoneme_labels)), replace = False, size = len(phoneme_labels))
    
    shuffled_images = images[index]
    shuffled_shape_labels = shape_labels[index]
    shuffled_position_labels = position_labels[index]
    shuffled_phoneme_labels = phoneme_labels[index]
    shuffled_second_phoneme_labels = second_phoneme_labels[index]
    return shuffled_images, shuffled_shape_labels, shuffled_position_labels, shuffled_phoneme_labels, shuffled_second_phoneme_labels

In [4]:
#Function to split data
def split_data(data, split_ratio=0.9):
    partition = round(len(data) * split_ratio)
    
    train = data[:partition]
    test = data[partition:]
    return train, test

In [5]:
#Function to prepare dataset to input network
def create_dataset(image_dir):
    dataset = []
    for image_name in os.listdir(image_dir):
        image = Image.open(image_dir + image_name)
        image_as_array = np.asarray(image)
        dataset.append(image_as_array)
    return np.array(dataset)

In [6]:
#Function to prepare dataset as grey images to input network
def create_grey_dataset(image_dir):
    grey_dataset = []
    for image_name in os.listdir(image_dir):
        image = Image.open(image_dir + image_name)
        image_as_array = np.asarray(image)
        r, g, b = image_as_array[:,:,0], image_as_array[:,:,1], image_as_array[:,:,2]
        grey_image = (0.3*r) + (0.59*g) +(0.11*b)
        grey_dataset.append(grey_image)
    return np.array(grey_dataset)

In [7]:
#Function to concatenate outputs of two networks into single vector
def join_outputs(out1, out2):
    joint_output = []
    for i in range(len(out1)):
        joint = np.concatenate((out1[i],out2[i]))
        joint_output.append(joint)
    return np.array(joint_output)

In [1]:
#Function to plot training and validation loss and accuracy
def plot_training_results(history, epochs = 5):
    
    loss=history.history['loss']
    val_loss=history.history['val_loss']
    
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']

    epochs_range = range(epochs)

    plt.figure(figsize=(8, 8))
    plt.subplot(1, 2, 1)
    plt.plot(epochs_range, loss, label='Training Loss')
    plt.plot(epochs_range, val_loss, label='Validation Loss')
    plt.legend(loc='upper right')
    plt.title('Training and Validation Loss')
    
    plt.subplot(1, 2, 2)
    plt.plot(epochs_range, acc, label='Training Accuracy')
    plt.plot(epochs_range, val_acc, label='Validation Accuracy')
    plt.legend(loc='lower right')
    plt.title('Training and Validation Accuracy')

    plt.show()
    return

In [8]:
#Open image folders
consonants_dir = os.path.join('/Users/User/Desktop/MSc Project/English Cued Speech/ConsonantImages/')
vowels_dir = os.path.join('/Users/User/Desktop/MSc Project/English Cued Speech/VowelImages/')
combined_dir = os.path.join('/Users/User/Desktop/MSc Project/English Cued Speech/PhonemeImages/')

#Open label files
consonant_labels = "/Users/User/Desktop/MSc Project/English Cued Speech/Labels_Consonant.txt"
vowel_labels = "/Users/User/Desktop/MSc Project/English Cued Speech/Labels_Vowel.txt"
combined_labels = "/Users/User/Desktop/MSc Project/English Cued Speech/Labels_All.txt"

In [9]:
#Extract labels
(vowel_shape_labels, vowel_position_labels, vowel_phoneme_labels, not_used) = extract_labels(vowel_labels)
(consonant_shape_labels, consonant_position_labels, consonant_phoneme_labels, or_this) = extract_labels(consonant_labels)
(combined_shape_labels, combined_position_labels, combined_consonant_labels, combined_vowel_labels) = extract_labels(combined_labels)

In [10]:
#Vowels Model

In [11]:
#Prepare images of vowels to input into network
vowel_images = create_dataset(vowels_dir)
vowel_images = vowel_images.reshape(len(vowel_images), 256, 256, 3)

In [12]:
#Randomly shuffle vowel images and labels
shuffled_vowels, shuffled_vowel_shape_labels, shuffled_vowel_position_labels, shuffled_vowel_phoneme_labels, non_used = shuffle_data(vowel_images, vowel_shape_labels, vowel_position_labels, vowel_phoneme_labels, not_used)

#Split training and test data
vowel_train, vowel_test = split_data(shuffled_vowels)
vowel_shape_labels_train, vowel_shape_labels_test = split_data(shuffled_vowel_shape_labels)
vowel_position_labels_train, vowel_position_labels_test = split_data(shuffled_vowel_position_labels)
vowel_phoneme_labels_train, vowel_phoneme_labels_test = split_data(shuffled_vowel_phoneme_labels)

In [13]:
#Build the network to recognise hand position
hand_position_model = tf.keras.models.Sequential([
  tf.keras.layers.experimental.preprocessing.Rescaling(1./255, input_shape=(256, 256, 3)),
  tf.keras.layers.Conv2D(8, (3, 3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Conv2D(8, (3, 3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2,2),
  tf.keras.layers.Conv2D(8, (3, 3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2,2),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(5, activation='softmax')
])

#Compile the network
hand_position_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

#Train the network
hand_position_model.fit(vowel_train, vowel_position_labels_train, epochs=5)

Epoch 1/5
33/33 [==============================] - 37s 1s/step - loss: 0.9243 - accuracy: 0.6586
Epoch 2/5
33/33 [==============================] - 37s 1s/step - loss: 0.2788 - accuracy: 0.9069
Epoch 3/5
33/33 [==============================] - 38s 1s/step - loss: 0.1680 - accuracy: 0.9564
Epoch 4/5
33/33 [==============================] - 38s 1s/step - loss: 0.1467 - accuracy: 0.9661
Epoch 5/5
33/33 [==============================] - 38s 1s/step - loss: 0.1159 - accuracy: 0.9699


In [14]:
#Test the network and print it's performance
hand_position_test_loss, hand_position_test_accuracy = hand_position_model.evaluate(vowel_test, vowel_position_labels_test)
print ('Test loss: {}, Test accuracy: {}'.format(hand_position_test_loss, hand_position_test_accuracy*100))

4/4 [==============================] - 1s 323ms/step - loss: 0.2910 - accuracy: 0.9739
Test loss: 0.2909955680370331, Test accuracy: 97.39130139350891


In [93]:
#Build network to recognise vowel phoneme
vowel_phoneme_model = tf.keras.models.Sequential([
  tf.keras.layers.experimental.preprocessing.Rescaling(1./255, input_shape=(256, 256, 3)),
  tf.keras.layers.Conv2D(8, (3, 3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Conv2D(8, (3, 3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2,2),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(13, activation='softmax')
])

#Complie the network
vowel_phoneme_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

#Train the network
vowel_phoneme_model.fit(vowel_train, vowel_phoneme_labels_train, epochs=5)

Epoch 1/5
33/33 [==============================] - 71s 2s/step - loss: 2.0246 - accuracy: 0.4035
Epoch 2/5
33/33 [==============================] - 21s 643ms/step - loss: 1.3260 - accuracy: 0.5761
Epoch 3/5
33/33 [==============================] - 22s 655ms/step - loss: 1.0437 - accuracy: 0.6178
Epoch 4/5
33/33 [==============================] - 22s 656ms/step - loss: 0.9208 - accuracy: 0.6382
Epoch 5/5
33/33 [==============================] - 21s 648ms/step - loss: 0.8423 - accuracy: 0.6683


In [94]:
#Test the network and print it's performance
vowel_phoneme_test_loss, vowel_phoneme_test_accuracy = vowel_phoneme_model.evaluate(vowel_test, vowel_phoneme_labels_test)
print ('Test loss: {}, Test accuracy: {}'.format(vowel_phoneme_test_loss, vowel_phoneme_test_accuracy*100))

4/4 [==============================] - 1s 216ms/step - loss: 0.8819 - accuracy: 0.6174
Test loss: 0.8819363117218018, Test accuracy: 61.739128828048706


In [95]:
#Obtain decisions from both networks and concatenate into a single decision vector
hand_position_decision = hand_position_model(vowel_train)

vowel_phoneme_decision = vowel_phoneme_model(vowel_train)

vowel_decision = join_outputs(hand_position_decision, vowel_phoneme_decision)

In [96]:
#Obtain test vectors to test network on
hand_position_decision_test = hand_position_model(vowel_test)

vowel_phoneme_decision_test = vowel_phoneme_model(vowel_test)

vowel_decision_test = join_outputs(hand_position_decision_test, vowel_phoneme_decision_test)

In [97]:
#Build network to make final decision of phoneme
vowel_decision_model = tf.keras.models.Sequential([
  tf.keras.layers.Dense(16, activation='relu'),
  tf.keras.layers.Dense(13, activation='softmax')
])

#Compile network
vowel_decision_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

#Train network
vowel_decision_model.fit(vowel_decision, vowel_phoneme_labels_train, epochs=30)

Epoch 1/30
33/33 [==============================] - 0s 4ms/step - loss: 2.6149 - accuracy: 0.0863
Epoch 2/30
33/33 [==============================] - 0s 2ms/step - loss: 2.4474 - accuracy: 0.3307
Epoch 3/30
33/33 [==============================] - 0s 3ms/step - loss: 2.2898 - accuracy: 0.4161
Epoch 4/30
33/33 [==============================] - 0s 2ms/step - loss: 2.1227 - accuracy: 0.4549
Epoch 5/30
33/33 [==============================] - 0s 1ms/step - loss: 1.9520 - accuracy: 0.6421
Epoch 6/30
33/33 [==============================] - 0s 1ms/step - loss: 1.7838 - accuracy: 0.6499
Epoch 7/30
33/33 [==============================] - 0s 1ms/step - loss: 1.6190 - accuracy: 0.6489
Epoch 8/30
33/33 [==============================] - 0s 1ms/step - loss: 1.4630 - accuracy: 0.6499
Epoch 9/30
33/33 [==============================] - 0s 2ms/step - loss: 1.3314 - accuracy: 0.6508
Epoch 10/30
33/33 [==============================] - 0s 1ms/step - loss: 1.2271 - accuracy: 0.6508
Epoch 11/30
33/33 [

In [98]:
#Test network and print result
vowel_test_loss, vowel_test_accuracy = vowel_decision_model.evaluate(vowel_decision_test, vowel_phoneme_labels_test)
print ('Test loss: {}, Test accuracy: {}'.format(vowel_test_loss, vowel_test_accuracy*100))

4/4 [==============================] - 0s 1ms/step - loss: 0.9441 - accuracy: 0.6087
Test loss: 0.9441329836845398, Test accuracy: 60.86956262588501


In [21]:
#Consonants Model

In [22]:
#Prepare images of consonants to input into network
consonant_images = create_dataset(consonants_dir)
consonant_images = consonant_images.reshape(len(consonant_images), 256, 256, 3)

In [23]:
#Randomly shuffle consonant images and labels
shuffled_consonants, shuffled_consonant_shape_labels, shuffled_consonant_position_labels, shuffled_consonant_phoneme_labels, nor_this = shuffle_data(consonant_images, consonant_shape_labels, consonant_position_labels, consonant_phoneme_labels, or_this)

#Split training and test data
consonant_train, consonant_test = split_data(shuffled_consonants)
consonant_shape_labels_train, consonant_shape_labels_test = split_data(shuffled_consonant_shape_labels)
consonant_position_labels_train, consonant_position_labels_test = split_data(shuffled_consonant_position_labels)
consonant_phoneme_labels_train, consonant_phoneme_labels_test = split_data(shuffled_consonant_phoneme_labels)

In [102]:
#Build the network to recognise hand shape
hand_shape_model = tf.keras.models.Sequential([
  tf.keras.layers.experimental.preprocessing.Rescaling(1./255, input_shape=(256, 256, 3)),
  tf.keras.layers.Conv2D(8, (3, 3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Conv2D(8, (3, 3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2,2),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(9, activation='softmax')
])

#Compile the network
hand_shape_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

#Train the network
hand_shape_model.fit(consonant_train, consonant_shape_labels_train, epochs=5)

Epoch 1/5
43/43 [==============================] - 30s 695ms/step - loss: 1.9746 - accuracy: 0.2424
Epoch 2/5
43/43 [==============================] - 28s 648ms/step - loss: 1.7619 - accuracy: 0.3632
Epoch 3/5
43/43 [==============================] - 28s 647ms/step - loss: 1.3836 - accuracy: 0.5337
Epoch 4/5
43/43 [==============================] - 29s 664ms/step - loss: 1.0310 - accuracy: 0.6464
Epoch 5/5
43/43 [==============================] - 30s 688ms/step - loss: 0.6363 - accuracy: 0.7902


In [103]:
#Test the network and print it's performance
hand_shape_test_loss, hand_shape_test_accuracy = hand_shape_model.evaluate(consonant_test, consonant_shape_labels_test)
print ('Test loss: {}, Test accuracy: {}'.format(hand_shape_test_loss, hand_shape_test_accuracy*100))

5/5 [==============================] - 1s 210ms/step - loss: 0.7781 - accuracy: 0.6933
Test loss: 0.7780782580375671, Test accuracy: 69.33333277702332


In [104]:
#Build network to recognise consonant phoneme
consonant_phoneme_model = tf.keras.models.Sequential([
  tf.keras.layers.experimental.preprocessing.Rescaling(1./255, input_shape=(256, 256, 3)),
  tf.keras.layers.Conv2D(8, (3, 3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Conv2D(8, (3, 3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2,2),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(25, activation='softmax')
])

#Complie the network
consonant_phoneme_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

#Train the network
consonant_phoneme_model.fit(consonant_train, consonant_phoneme_labels_train, epochs=5)

Epoch 1/5
43/43 [==============================] - 32s 733ms/step - loss: 3.0141 - accuracy: 0.1105
Epoch 2/5
43/43 [==============================] - 30s 696ms/step - loss: 2.8448 - accuracy: 0.1349
Epoch 3/5
43/43 [==============================] - 28s 657ms/step - loss: 2.4961 - accuracy: 0.3099
Epoch 4/5
43/43 [==============================] - 28s 648ms/step - loss: 1.9902 - accuracy: 0.4277
Epoch 5/5
43/43 [==============================] - 28s 656ms/step - loss: 1.5418 - accuracy: 0.5182


In [105]:
#Test the network and print it's performance
consonant_phoneme_test_loss, consonant_phoneme_test_accuracy = consonant_phoneme_model.evaluate(consonant_test, consonant_phoneme_labels_test)
print ('Test loss: {}, Test accuracy: {}'.format(consonant_phoneme_test_loss, consonant_phoneme_test_accuracy*100))

5/5 [==============================] - 1s 200ms/step - loss: 1.7762 - accuracy: 0.4200
Test loss: 1.7762054204940796, Test accuracy: 41.999998688697815


In [106]:
#Obtain decisions from both networks and concatenate into a single decision vector
hand_shape_decision = hand_shape_model(consonant_train)

consonant_phoneme_decision = consonant_phoneme_model(consonant_train)

consonant_decision = join_outputs(hand_shape_decision, consonant_phoneme_decision)

In [107]:
#Obtain test vectors to test network on
hand_shape_decision_test = hand_shape_model(consonant_test)

consonant_phoneme_decision_test = consonant_phoneme_model(consonant_test)

consonant_decision_test = join_outputs(hand_shape_decision_test, consonant_phoneme_decision_test)

In [108]:
#Build network to make final decision of phoneme
consonant_decision_model = tf.keras.models.Sequential([
  tf.keras.layers.Dense(16, activation='relu'),
  tf.keras.layers.Dense(25, activation='softmax')
])

#Compile network
consonant_decision_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

#Train network
consonant_decision_model.fit(consonant_decision, consonant_phoneme_labels_train, epochs=30)

Epoch 1/30
43/43 [==============================] - 0s 1ms/step - loss: 3.1343 - accuracy: 0.1045
Epoch 2/30
43/43 [==============================] - 0s 1ms/step - loss: 3.0027 - accuracy: 0.1913
Epoch 3/30
43/43 [==============================] - 0s 1ms/step - loss: 2.8366 - accuracy: 0.2557
Epoch 4/30
43/43 [==============================] - 0s 2ms/step - loss: 2.6523 - accuracy: 0.3432
Epoch 5/30
43/43 [==============================] - 0s 2ms/step - loss: 2.4730 - accuracy: 0.3892
Epoch 6/30
43/43 [==============================] - 0s 1ms/step - loss: 2.3043 - accuracy: 0.4122
Epoch 7/30
43/43 [==============================] - 0s 1ms/step - loss: 2.1463 - accuracy: 0.4085
Epoch 8/30
43/43 [==============================] - 0s 1ms/step - loss: 1.9999 - accuracy: 0.4299
Epoch 9/30
43/43 [==============================] - 0s 1ms/step - loss: 1.8680 - accuracy: 0.4507
Epoch 10/30
43/43 [==============================] - 0s 1ms/step - loss: 1.7536 - accuracy: 0.4885
Epoch 11/30
43/43 [

In [109]:
#Test network and print result
consonant_test_loss, consonant_test_accuracy = consonant_decision_model.evaluate(consonant_decision_test, consonant_phoneme_labels_test)
print ('Test loss: {}, Test accuracy: {}'.format(consonant_test_loss, consonant_test_accuracy*100))

5/5 [==============================] - 0s 1ms/step - loss: 1.3508 - accuracy: 0.5133
Test loss: 1.3507564067840576, Test accuracy: 51.33333206176758


In [32]:
#Grey Vowels Model

In [72]:
#Convert images of consonants to greyscale and prepare to input into network
grey_vowel_images = create_grey_dataset(vowels_dir)
grey_vowel_images = grey_vowel_images.reshape(len(grey_vowel_images), 256, 256, 1)

In [73]:
#Randomly shuffle consonant images and labels
grey_shuffled_vowels, grey_shuffled_vowel_shape_labels, grey_shuffled_vowel_position_labels, grey_shuffled_vowel_phoneme_labels, grey_non_used = shuffle_data(grey_vowel_images, vowel_shape_labels, vowel_position_labels, vowel_phoneme_labels, not_used)

#Split training and test data
grey_vowel_train, grey_vowel_test = split_data(grey_shuffled_vowels)
grey_vowel_shape_labels_train, grey_vowel_shape_labels_test = split_data(grey_shuffled_vowel_shape_labels)
grey_vowel_position_labels_train, grey_vowel_position_labels_test = split_data(grey_shuffled_vowel_position_labels)
grey_vowel_phoneme_labels_train, grey_vowel_phoneme_labels_test = split_data(grey_shuffled_vowel_phoneme_labels)

In [74]:
#Build the network to recognise hand position
grey_hand_position_model = tf.keras.models.Sequential([
  tf.keras.layers.experimental.preprocessing.Rescaling(1./255, input_shape=(256, 256, 1)),
  tf.keras.layers.Conv2D(8, (3, 3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Conv2D(8, (3, 3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2,2),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(9, activation='softmax')
])

#Compile the networ
grey_hand_position_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

#Train the network
grey_hand_position_model.fit(grey_vowel_train, grey_vowel_position_labels_train, epochs=5)

Epoch 1/5
33/33 [==============================] - 32s 961ms/step - loss: 0.6519 - accuracy: 0.7672
Epoch 2/5
33/33 [==============================] - 31s 948ms/step - loss: 0.2453 - accuracy: 0.9243
Epoch 3/5
33/33 [==============================] - 32s 957ms/step - loss: 0.1449 - accuracy: 0.9631
Epoch 4/5
33/33 [==============================] - 30s 910ms/step - loss: 0.1096 - accuracy: 0.9719
Epoch 5/5
33/33 [==============================] - 32s 964ms/step - loss: 0.0863 - accuracy: 0.9835


In [75]:
#Test the network and print it's performance
grey_hand_position_test_loss, grey_hand_position_test_accuracy = grey_hand_position_model.evaluate(grey_vowel_test, grey_vowel_position_labels_test)
print ('Test loss: {}, Test accuracy: {}'.format(grey_hand_position_test_loss, grey_hand_position_test_accuracy*100))

4/4 [==============================] - 1s 237ms/step - loss: 0.1382 - accuracy: 0.9826
Test loss: 0.1381930559873581, Test accuracy: 98.26086759567261


In [76]:
#Build network to recognise vowel phoneme
grey_vowel_phoneme_model = tf.keras.models.Sequential([
  tf.keras.layers.experimental.preprocessing.Rescaling(1./255, input_shape=(256, 256, 1)),
  tf.keras.layers.Conv2D(8, (3, 3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Conv2D(8, (3, 3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2,2),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(25, activation='softmax')
])

#Complie the network
grey_vowel_phoneme_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

#Train the network
grey_vowel_phoneme_model.fit(grey_vowel_train, grey_vowel_phoneme_labels_train, epochs=5)

Epoch 1/5
33/33 [==============================] - 32s 969ms/step - loss: 1.8379 - accuracy: 0.5063
Epoch 2/5
33/33 [==============================] - 31s 930ms/step - loss: 1.1203 - accuracy: 0.6237
Epoch 3/5
33/33 [==============================] - 30s 915ms/step - loss: 0.9301 - accuracy: 0.6625
Epoch 4/5
33/33 [==============================] - 31s 929ms/step - loss: 0.8245 - accuracy: 0.7129
Epoch 5/5
33/33 [==============================] - 30s 917ms/step - loss: 0.7351 - accuracy: 0.7139


In [77]:
#Test the network and print it's performance
grey_vowel_phoneme_test_loss, grey_vowel_phoneme_test_accuracy = grey_vowel_phoneme_model.evaluate(grey_vowel_test, grey_vowel_phoneme_labels_test)
print ('Test loss: {}, Test accuracy: {}'.format(grey_vowel_phoneme_test_loss, grey_vowel_phoneme_test_accuracy*100))

4/4 [==============================] - 1s 254ms/step - loss: 1.0889 - accuracy: 0.5913
Test loss: 1.0889414548873901, Test accuracy: 59.130436182022095


In [78]:
#Obtain decisions from both networks and concatenate into a single decision vector
grey_hand_position_decision = grey_hand_position_model(grey_vowel_train)

grey_vowel_phoneme_decision = grey_vowel_phoneme_model(grey_vowel_train)

grey_vowel_decision = join_outputs(grey_hand_position_decision, grey_vowel_phoneme_decision)

In [79]:
#Obtain test vectors to test network on
grey_hand_position_decision_test = grey_hand_position_model(grey_vowel_test)

grey_vowel_phoneme_decision_test = grey_vowel_phoneme_model(grey_vowel_test)

grey_vowel_decision_test = join_outputs(grey_hand_position_decision_test, grey_vowel_phoneme_decision_test)

In [80]:
#Build network to make final decision of phoneme
grey_vowel_decision_model = tf.keras.models.Sequential([
  tf.keras.layers.Dense(16, activation='relu'),
  tf.keras.layers.Dense(25, activation='softmax')
])

#Compile network
grey_vowel_decision_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

#Train network
grey_vowel_decision_model.fit(grey_vowel_decision, grey_vowel_phoneme_labels_train, epochs=30)

Epoch 1/30
33/33 [==============================] - 0s 2ms/step - loss: 3.1442 - accuracy: 0.0873
Epoch 2/30
33/33 [==============================] - 0s 2ms/step - loss: 2.9739 - accuracy: 0.2357
Epoch 3/30
33/33 [==============================] - 0s 1ms/step - loss: 2.7386 - accuracy: 0.4607
Epoch 4/30
33/33 [==============================] - 0s 1ms/step - loss: 2.4316 - accuracy: 0.4597
Epoch 5/30
33/33 [==============================] - 0s 1ms/step - loss: 2.0889 - accuracy: 0.5199
Epoch 6/30
33/33 [==============================] - 0s 1ms/step - loss: 1.7834 - accuracy: 0.6295
Epoch 7/30
33/33 [==============================] - 0s 2ms/step - loss: 1.5529 - accuracy: 0.6440
Epoch 8/30
33/33 [==============================] - 0s 2ms/step - loss: 1.3922 - accuracy: 0.6499
Epoch 9/30
33/33 [==============================] - 0s 1ms/step - loss: 1.2752 - accuracy: 0.6508
Epoch 10/30
33/33 [==============================] - 0s 2ms/step - loss: 1.1879 - accuracy: 0.6518
Epoch 11/30
33/33 [

In [81]:
#Test network and print result
grey_vowel_test_loss, grey_vowel_test_accuracy = grey_vowel_decision_model.evaluate(grey_vowel_decision_test, grey_vowel_phoneme_labels_test)
print ('Test loss: {}, Test accuracy: {}'.format(grey_vowel_test_loss, grey_vowel_test_accuracy*100))

4/4 [==============================] - 0s 3ms/step - loss: 0.9820 - accuracy: 0.6000
Test loss: 0.9820446372032166, Test accuracy: 60.00000238418579


In [43]:
#Grey Consonants Model

In [44]:
#Convert images of consonants to greyscale and prepare to input into network
grey_consonant_images = create_grey_dataset(consonants_dir)
grey_consonant_images = grey_consonant_images.reshape(len(grey_consonant_images), 256, 256, 1)

In [45]:
#Randomly shuffle consonant images and labels
grey_shuffled_consonants, grey_shuffled_consonant_shape_labels, grey_shuffled_consonant_position_labels, grey_shuffled_consonant_phoneme_labels, grey_nor_this = shuffle_data(grey_consonant_images, consonant_shape_labels, consonant_position_labels, consonant_phoneme_labels, or_this)

#Split training and test data
grey_consonant_train, grey_consonant_test = split_data(grey_shuffled_consonants)
grey_consonant_shape_labels_train, grey_consonant_shape_labels_test = split_data(grey_shuffled_consonant_shape_labels)
grey_consonant_position_labels_train, grey_consonant_position_labels_test = split_data(grey_shuffled_consonant_position_labels)
grey_consonant_phoneme_labels_train, grey_consonant_phoneme_labels_test = split_data(grey_shuffled_consonant_phoneme_labels)

In [83]:
#Build the network to recognise hand shape
grey_hand_shape_model = tf.keras.models.Sequential([
  tf.keras.layers.experimental.preprocessing.Rescaling(1./255, input_shape=(256, 256, 1)),
  tf.keras.layers.Conv2D(8, (3, 3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Conv2D(8, (3, 3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2,2),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(9, activation='softmax')
])

#Compile the networ
grey_hand_shape_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

#Train the network
grey_hand_shape_model.fit(grey_consonant_train, grey_consonant_shape_labels_train, epochs=5)

Epoch 1/5
43/43 [==============================] - 40s 936ms/step - loss: 1.9867 - accuracy: 0.2246
Epoch 2/5
43/43 [==============================] - 41s 948ms/step - loss: 1.7503 - accuracy: 0.3684
Epoch 3/5
43/43 [==============================] - 40s 933ms/step - loss: 1.4320 - accuracy: 0.5226
Epoch 4/5
43/43 [==============================] - 41s 949ms/step - loss: 1.1139 - accuracy: 0.6368
Epoch 5/5
43/43 [==============================] - 41s 955ms/step - loss: 0.8265 - accuracy: 0.7354


In [84]:
#Test the network and print it's performance
grey_hand_shape_test_loss, grey_hand_shape_test_accuracy = grey_hand_shape_model.evaluate(grey_consonant_test, grey_consonant_shape_labels_test)
print ('Test loss: {}, Test accuracy: {}'.format(grey_hand_shape_test_loss, grey_hand_shape_test_accuracy*100))

5/5 [==============================] - 1s 271ms/step - loss: 1.0187 - accuracy: 0.6333
Test loss: 1.0186818838119507, Test accuracy: 63.333332538604736


In [48]:
#Build network to recognise consonant phoneme
grey_consonant_phoneme_model = tf.keras.models.Sequential([
  tf.keras.layers.experimental.preprocessing.Rescaling(1./255, input_shape=(256, 256, 1)),
  tf.keras.layers.Conv2D(8, (3, 3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Conv2D(8, (3, 3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2,2),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(25, activation='softmax')
])

#Complie the network
grey_consonant_phoneme_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

#Train the network
grey_consonant_phoneme_model.fit(grey_consonant_train, grey_consonant_phoneme_labels_train, epochs=5)

Epoch 1/5
43/43 [==============================] - 40s 928ms/step - loss: 2.9621 - accuracy: 0.1119
Epoch 2/5
43/43 [==============================] - 41s 942ms/step - loss: 2.6837 - accuracy: 0.2187
Epoch 3/5
43/43 [==============================] - 40s 924ms/step - loss: 2.4077 - accuracy: 0.3262
Epoch 4/5
43/43 [==============================] - 40s 940ms/step - loss: 2.0682 - accuracy: 0.4114
Epoch 5/5
43/43 [==============================] - 41s 946ms/step - loss: 1.7353 - accuracy: 0.4885


In [49]:
#Test the network and print it's performance
grey_consonant_phoneme_test_loss, grey_consonant_phoneme_test_accuracy = grey_consonant_phoneme_model.evaluate(grey_consonant_test, grey_consonant_phoneme_labels_test)
print ('Test loss: {}, Test accuracy: {}'.format(grey_consonant_phoneme_test_loss, grey_consonant_phoneme_test_accuracy*100))

5/5 [==============================] - 1s 270ms/step - loss: 2.1323 - accuracy: 0.3667
Test loss: 2.132263660430908, Test accuracy: 36.666667461395264


In [50]:
#Obtain decisions from both networks and concatenate into a single decision vector
grey_hand_shape_decision = grey_hand_shape_model(grey_consonant_train)

grey_consonant_phoneme_decision = grey_consonant_phoneme_model(grey_consonant_train)

grey_consonant_decision = join_outputs(grey_hand_shape_decision, grey_consonant_phoneme_decision)

In [51]:
#Obtain test vectors to test network on
grey_hand_shape_decision_test = grey_hand_shape_model(grey_consonant_test)

grey_consonant_phoneme_decision_test = grey_consonant_phoneme_model(grey_consonant_test)

grey_consonant_decision_test = join_outputs(grey_hand_shape_decision_test, grey_consonant_phoneme_decision_test)

In [52]:
#Build network to make final decision of phoneme
grey_consonant_decision_model = tf.keras.models.Sequential([
  tf.keras.layers.Dense(16, activation='relu'),
  tf.keras.layers.Dense(25, activation='softmax')
])

#Compile network
grey_consonant_decision_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

#Train network
grey_consonant_decision_model.fit(grey_consonant_decision, grey_consonant_phoneme_labels_train, epochs=30)

Epoch 1/30
43/43 [==============================] - 0s 2ms/step - loss: 3.1596 - accuracy: 0.0726
Epoch 2/30
43/43 [==============================] - 0s 2ms/step - loss: 3.0519 - accuracy: 0.1534
Epoch 3/30
43/43 [==============================] - 0s 2ms/step - loss: 2.9128 - accuracy: 0.2380
Epoch 4/30
43/43 [==============================] - 0s 2ms/step - loss: 2.7441 - accuracy: 0.3269
Epoch 5/30
43/43 [==============================] - 0s 2ms/step - loss: 2.5669 - accuracy: 0.4047
Epoch 6/30
43/43 [==============================] - 0s 2ms/step - loss: 2.3940 - accuracy: 0.4211
Epoch 7/30
43/43 [==============================] - 0s 2ms/step - loss: 2.2329 - accuracy: 0.4344
Epoch 8/30
43/43 [==============================] - 0s 2ms/step - loss: 2.0910 - accuracy: 0.4255
Epoch 9/30
43/43 [==============================] - 0s 2ms/step - loss: 1.9708 - accuracy: 0.4618
Epoch 10/30
43/43 [==============================] - 0s 2ms/step - loss: 1.8716 - accuracy: 0.4685
Epoch 11/30
43/43 [

In [53]:
#Test network and print result
grey_consonant_test_loss, grey_consonant_test_accuracy = grey_consonant_decision_model.evaluate(grey_consonant_decision_test, grey_consonant_phoneme_labels_test)
print ('Test loss: {}, Test accuracy: {}'.format(grey_consonant_test_loss, grey_consonant_test_accuracy*100))

5/5 [==============================] - 0s 2ms/step - loss: 1.7051 - accuracy: 0.4200
Test loss: 1.70505952835083, Test accuracy: 41.999998688697815


In [99]:
#Build SVM classifier for vowel
vowel_svm = svm.SVC(kernel='linear')

vowel_svm.fit(vowel_decision, vowel_phoneme_labels_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [100]:
#Test SVM classifier and print accuracy
final_vowel_decision = vowel_svm.predict(vowel_decision_test)

print("Accuracy:",metrics.accuracy_score(vowel_phoneme_labels_test, final_vowel_decision)*100)

In [110]:
#Build SVM classifier for consonant
consonant_svm = svm.SVC(kernel='linear')

consonant_svm.fit(consonant_decision, consonant_phoneme_labels_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [111]:
#Test SVM classifier and print accuracy
final_consonant_decision = consonant_svm.predict(consonant_decision_test)

print("Accuracy:",metrics.accuracy_score(consonant_phoneme_labels_test, final_consonant_decision)*100)

In [57]:
#Build Naive-Bayes classifier for vowel
vowel_nb = naive_bayes.GaussianNB()

vowel_nb.fit(vowel_decision, vowel_phoneme_labels_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [58]:
#Test Naive-Bayes classifier and print accuracy
bae_vowel_decision = vowel_nb.predict(vowel_decision_test)

print("Accuracy:",metrics.accuracy_score(vowel_phoneme_labels_test, bae_vowel_decision)*100)

In [63]:
#Build Naive-Bayes classifier for consonant
consonant_nb = naive_bayes.GaussianNB()

consonant_nb.fit(consonant_decision, consonant_phoneme_labels_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [66]:
#Test Naive-Bayes classifier and print accuracy

bae_consonant_decision = consonant_nb.predict(consonant_decision_test)

print("Accuracy:",metrics.accuracy_score(consonant_phoneme_labels_test, bae_consonant_decision)*100)